# 2017-04-09 : learning about networking using the zmq python library
----
Reminders : I'm trying to write a pythn-on script that run as background (daemon) on the Pi, that allow to interact with few command lines (API, maybe using zmq and/or socket) to do 3 tasks :
- Ask the camera if it's ready to record
- Allow the camera to answer positively or negatively
- Ask the camera to cut down the connection  


- [ ] Create a daemon : Canceled (Don't need one anymore)

Like last week, I have connection problems to the Pi. But this time, restarting it didn't solved the problem. After talking with L. Perrinet, we didn't found where it come from.  
I'll work on the daemon/API script until further notice.  
More informations about zmq : http://zeromq.org/, http://api.zeromq.org/ and http://zguide.zeromq.org/py:all  
Going further with the interactive python scripts : https://docs.python.org/2/library/thread.html#module-thread and https://docs.python.org/2/library/threading.html  

The connection to the Pi works, it go a new IP : __10.164.7.213__

    ssh pi@10.164.7.213

To learn how zmq is working, I'm reading the (complete) documentation : http://zguide.zeromq.org/py:all  
The zmq python library isn't installed on the pi. Problem : When trying to install it through

    sudo aptitude install python3-zmq

output :

    E: Could not get lock /var/lib/dpkg/lock - open (11: Resource temporarily unavailable)
    E: Unable to lock the administration directory (/var/lib/dpkg/), is another process using it?
    W: Could not lock the cache file; this usually means that dpkg or another apt tool is already installing packages.
    (...)
    
But no other apt process should be running.

    ps aux | grep apt

output 3 apt process running :

    root       747  0.0  0.3   6184  2956 pts/0    S+   14:26   0:00 sudo apt-get upgrade
    root       751  7.3  3.4  33656 30592 pts/0    S+   14:26   3:32 apt-get upgrade
    pi       13293  0.0  0.1   4248  1432 pts/2    S+   15:14   0:00 grep --color=auto apt
    
After verification, they were launched by L. Perrinet. Waiting their end to install zmq.  
zmq python library is succesfully installed on the pi.

The following scripts works on a local machine but doesn't initiate a computer/pi connection :

    '''Server side'''

    import zmq

    context = zmq.Context()
    print('Connecting to client...')
    socket = context.socket(zmq.REQ)
    socket.connect("tcp://localhost:5555")
    print('Client connected')

    for request in range(10):
        print("Sending request %s..." % request)
        socket.send(b"Hello")

        message = socket.recv() #Wait for next request from client
        print("Received reply %s [ %s ]" % (request, message))

    print('No more requests')
  

    '''Client side'''

    import time
    import zmq

    context = zmq.Context()
    socket = context.socket(zmq.REP)
    socket.bind("tcp://*:5555")
    print('Server connected')

    while True:
        message = socket.recv() #Wait for next request from client
        print("Received request: %s" % message)

        time.sleep(2)
        socket.send(b"World") #Send reply back to client
        
More informations about print(b"sting") : http://stackoverflow.com/questions/6269765/what-does-the-b-character-do-in-front-of-a-string-literal. It allow the string to be converted into bytes, necessary to be usable for other programming language.  

To make these scripts works with a computer/pi connection, we need to modify the socket.connect parameters, from

    socket.connect("tcp://localhost:5555")
    
to 

    socket.connect("tcp://10.164.7.213:5555")
    
Then pi output :

    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    
While computer output :

    Connecting to client...
    Client connected
    Sending request 0...
    Received reply 0 [ b'World' ]
    Sending request 1...
    Received reply 1 [ b'World' ]
    Sending request 2...
    Received reply 2 [ b'World' ]
    Sending request 3...
    Received reply 3 [ b'World' ]
    Sending request 4...
    Received reply 4 [ b'World' ]
    Sending request 5...
    Received reply 5 [ b'World' ]
    Sending request 6...
    Received reply 6 [ b'World' ]
    Sending request 7...
    Received reply 7 [ b'World' ]
    Sending request 8...
    Received reply 8 [ b'World' ]
    Sending request 9...
    Received reply 9 [ b'World' ]
    No more requests

There can be only one zmq context per process. It'll contain all the sockets of that process. Wrinting two contexts in one script means there's two process from zmq's point of view.  
zmq interact with sockets by applying modifications : open/close, set option/get option, bind/connect and send/receive (recv) messages.  
Client and server use different commands under certain circumstances. __Client__ is usually the most __dynamic node__, and __connect__ its sockets while __server__ is the most __static one__ and __bind__ its sockets, because it have the most stable adress. Here server will be the Pi because it is central to our network (we want to communicate with it from multiple computers).  
zmq allows to start each node independantly, contrary to other network managers that need the server to start first.  
Sockets are open until other notice and once bind/connected, automaticaly accept to send/receive messages (security?). 

Trying to add the picamera interaction into the zmq scripts...  
A problem with the following Pi-sided line seems to induce problems :

    socket.send(struct.pack('<L', stream.tell()))

# 2017-05-10 : merging the zmq and picamera scripts
----

- [X] Solve the file format problem

The computer-sided script output : 

    Received reply 0 [ b'\xf0\xd9\x02\x00' ]
    (...)
    ---> 22     image = Image.open(image_stream)
    (...)
    OSError: cannot identify image file <_io.BytesIO object at 0x7f8417cb8af0>

Reminds me of a previous problem I had with a file format problem : see W2_Progress_and_notes.ipynb/2017-05-05  
But this time, the error is triggered from the first image. The problem may come from a encoding problem at the pi side.  

The Pi-sided script output :

    ---> socket.send(struct.pack('<L', stream.tell()))
    File "socket.pyx", line 574, in zmq.backend.cython.socket.Socket.send (zmq/backend/cython/socket.c:5390)
    File "socket.pyx", line 621, in zmq.backend.cython.socket.Socket.send (zmq/backend/cython/socket.c:5157)
    File "socket.pyx", line 181, in zmq.backend.cython.socket._send_copy (zmq/backend/cython/socket.c:2110)
    File "checkrc.pxd", line 21, in zmq.backend.cython.checkrc._check_rc (zmq/backend/cython/socket.c:6103)
    zmq.error.ZMQError: Operation cannot be accomplished in current state

The error may come from an error in the script.  
After some researches, it is a common bug and may be caused by "deadlocks" (see : http://stackoverflow.com/questions/41009900/python-zmq-operation-cannot-be-accomplished-in-current-state)  
As suggested here : http://stackoverflow.com/questions/36391495/deadlock-when-synchronizing-two-simple-python3-scripts-using-0mq-zeromq, I tried to close sockets and contexts, but it did not solved the problem.  

After several hours of research, I'm still unable to solve that bug...  

I tried to get rid of loops inside the Pi-sided script to simplify it, because I was wondering that zmq could only send one answer per request.  
I might be right, because the Pi-sided error message is not showing anymore.  

Now I have a problem with the data that the Pi is sending, as the computer-sided script is outputing :

    (...)
    Sending request 0...
    Received reply 0 [ b'' ]
    (...)
    ---> 25     image = Image.open(image_stream)
    (...)
    OSError: cannot identify image file <_io.BytesIO object at 0x7f10dce2e728>
    
So the Pi reply seems to be an empty bytes string, instead of containing the bytes string component the captured image.

- [X] Solve the empty string reply problem

Scripts modified a bit to send 5 requests from computer and get 5 replies from Pi, and integrated the 'kill yourself' signal that make the Pi cut down the connection. No error except for the empty strings I get as reply. 

Modifying the pi-sided script to show the picture values through 

    print('dbkey n1', stream.getvalue())
    
Allow to see that picture are taken and the stream contain bytes strings. Why are these not sent correclty to the computer?  
The data may be too big to be sent entirely? 

    print('dbkey n1', len(stream.getvalue()))
    
at Pi, output : 

    (...)
    ('dbkey n1', 204193)
    (...)

while at computer, output :

    (...)
    (dbkey n1, 0)
    
More precisly, the byte strings seems to be lost in the Pi-sided script. See :

    (...)
    print(len(stream.getvalue()))
    (...)
    print(len(stream))
    (...)
    print(len(stream.read1(-1)))

# 2017-05-11
------
Still need to solve the empty string reply problem...  
To resume what I know : the communication through zmq network seem to work correctly, as messages are transmitted from computer to Pi and vice versa. When asked, the picamera take pictures, as shown by :

    stream = io.BytesIO()
    (...)
    camera.capture(stream, format='jpeg')
	print('dbkey n1', len(stream.getvalue()))
    (...)
    
output : 

    ('dbkey n1', 199757)
    
but just after these lines : 

    (...)
	reply = stream.read1(-1)
	print('dbkey n3', len(reply))

	socket.send(reply)
    (...)
    
output :

    ('dbkey n3', 0)

So the problem is created among these lines.  
The error was stupid. After working a day on it, I just replaced 
    
    (...)
    reply = stream.read1(-1)
    (...)

by

    (...)
    reply = stream.getvalue()
    (...)
    
and it finally works.  
The capture colors are a bit weird. Close objects looks fine but some distants objects looks pink instead of purple. It may be interesting to play with camera's saturation and white balance parameters to correct these colors (as suggested here : https://raspberrypi.stackexchange.com/questions/14515/raspi-cam-color-of-image).

# outline
------


